<p style="text-align: center;" ><font size="+3"><u><b>Laboratory 4 Part 2: System Identification and Optimization</u></b></p>

<p style="text-align: left;" ><font size="+1"><b>Group #: ___________   Team Members: _______________  </b></p>

<p style="text-align: left;" ><font size="+1"><b>Optimization Basics</b></p>

<div class="alert alert-block alert-warning">
<font color=black>

When you define an optimization problem you must consider three main elements: <br/>
1) the **optimization parameters**: the variables you are trying to find, <br/>
2) the **objective function**: a function (of the optimization parameters) of which you are trying to find minimum (or maximum); this is often a fitting function (e.g. a measure of error in a data fit), and <br/>
3) the **optimization algorithm/optimizer** – which is the particular algorithm we are using to find the optimum. Mathematically, we may write the optimization as
\begin{equation}
x^{*} = \underset{x}{\operatorname{argmin}}  f(x)
\end{equation} 
    
where $x$ is a vector of optimization parameters and $f(x)$ is the objective function. The minimization is defined implicitly and needs to be solved using some algorithm. <br/>
    
An example of how to code a simple optimization problem in Python is provided on Blackboard (`Lab4 Optimization Example.ipynb`). Run the code and understand the output, what the code is doing, and how to formulate a general optimization problem with the help of descriptions provided below.
</font> 
</div>

<p style="text-align: left;" ><font size="+1"><b>System Identification in Simulated Data</b></p>

<div class="alert alert-block alert-warning">
<font color=black>

We are going to use the same kind of procedure as outlined above to find out certain parameters in a physical microscope system. To do this we need a functional form of our data simulator. Specifically, you need to convert the simulation code you wrote from Part 1 of this lab to a function called `make_psf`. The inputs to the function are radius of the exit pupil (`r`), coefficients to the Zernike polynomials (`coefs`) that define any system aberrations, the corresponding mode indices of the Zernike polynomials (`mode_indices`), and size of the PSF you want to make (`nxy`). The output of the function should be a simulated PSF. For example, you should be able to execute the following command:
    
`def make_psf(r, coefs, mode_indices, nxy):` <br/>
`    # code block computing psf`<br/>
`    return psf`<br/>

Call the function and confirm you get the same results as before. <br/>
    
Next, we’ll write an objective function. Create a function in Python called `objfunc`. The inputs to the function are:
1) the optimization parameters in vector form (`xv`). Note that all the parameters need to be in a single vector, and the vector needs to be the first argument to the objective function
2) the PSF you want to fit to (a simulated PSF in this case, or the measured PSF in real data, `mpsf`), and 
3) the mode indices of the Zernike polynomials (`mode_indices`) <br/>
    
The objective function to the optimization is sum of squared error between the estimated PSF and PSF you want to fit to (`mpsf`). In a separate script, write the objective function with the following syntax:

`def objfunc(xv, mpsf, mode_indices):` <br/>
`    # code block computing the value of the objective function` <br/>
`    return val` <br/>

Note that our model did not account for the magnitude of the PSF. Before calculating the sum of squared error, normalize both PSFs to the same scale, e.g., between 0 and 1. <br/>

In your optimization routine, similar to the example in `Lab4 Optimization Example.ipynb`, display a figure containing `mpsf`, the estimated PSF (from `make_psf`), and the absolute difference between the two with a colorbar. This will allow you to see how the optimization is doing in real time. <br/><br/>

</font> 
</div>

<div class="alert alert-block alert-success">
<font color=black>

1. In summary, your objective function should contain the following three parts:

- Parse `xv`
- Generate model prediction of PSF using parsed `xv` 
- Normalize `mpsf` and the PSF from 2) and calculate and return the sum of squared error between the two (e.g. over all pixels)

Show your code below:

</font> 
</div>

In [ ]:
import numpy as np
from zernike import zernike 
import cma
import matplotlib.pyplot as plt

def make_psf(r, coefs, mode_indices, nxy):
    lambd = 540E-6  # setup
    zi = 300        # setup
    sampling = 0.2
    x = np.linspace(-1, 1, nxy)
    X, Y = np.meshgrid(x, x)

    fx = np.fft.fftshift(np.fft.fftfreq(nxy, d=sampling))
    fy = np.fft.fftshift(np.fft.fftfreq(nxy, d=sampling))

    FX, FY = np.meshgrid(-lambd*zi*fx, -lambd*zi*fy)
    H_amp = (FX**2 + FY**2) < r**2

    z_sum = zernike(-lambd * zi * fx / r, -lambd * zi * fy / r, mode_indices, coefs)
    phase = np.exp(-1j * 2 * np.pi * z_sum)

    H = H_amp * phase

    h = np.fft.ifftshift(np.fft.ifft2(np.fft.fftshift(H)))
    psf = np.abs(h)**2
    psf /= psf.sum()

    return psf


def objfunc(xv, mpsf, mode_indices):
    r = 1 
    nxy = mpsf.shape[0]
    est_psf = make_psf(r, xv, mode_indices, nxy)

    mpsf_norm = (mpsf - mpsf.min()) / (mpsf.max() - mpsf.min())
    est_psf_norm = (est_psf - est_psf.min()) / (est_psf.max() - est_psf.min())

    val = np.sum((mpsf_norm - est_psf_norm) ** 2)
    return val


In [6]:
import matplotlib.pyplot as plt

def show_optimization_result(mpsf, est_psf):
    diff = np.abs(mpsf - est_psf)

    fig, axs = plt.subplots(1, 3, figsize=(12, 4))
    axs[0].imshow(mpsf, cmap='coolwarm')
    axs[0].set_title('Measured PSF')
    axs[1].imshow(est_psf, cmap='coolwarm')
    axs[1].set_title('Estimated PSF')
    im = axs[2].imshow(diff, cmap='coolwarm')
    axs[2].set_title('Difference')
    fig.colorbar(im, ax=axs[2])
    plt.tight_layout()
    plt.show()


<div class="alert alert-block alert-success">
<font color=black>

2. Use your `make_psf` function to generate a PSF from part 1 of this lab, for example: 
    
`mode_indices = [7, 8]`<br/>
`coefs = [0.05, -0.2]`<br/>

Assume that you know the Zernike modes but do not know the coefficients, use the optimization routine to see if you can recover the coefficients.
    
A few tricks to help the optimization:
- Have good initial guesses (close to the final solution) are important
- When computing the objective function, you can zoom in to a region of interest containing the PSF instead of calculating sum of squared error over the entire image <br/>
- Show an image of both PSFs and their absolute difference

</font> 
</div>

In [ ]:
x0 = [0.1, -0.1]
sigma0 = 0.05
opts = {'tolfun': 1e-8, 'tolx': 1e-5, 'popsize': 10, 'verb_disp': 1}

es = cma.CMAEvolutionStrategy(x0, sigma0, opts)
fit = np.empty(opts['popsize'])

while not es.stop():
    new_solutions = es.ask()
    for i, sol in enumerate(new_solutions):
        fit[i] = objfunc(sol, mpsf, mode_indices)
    es.tell(new_solutions, fit)
    es.disp(1)

xopt = es.result[0]
print("Recovered coefficients:", xopt)

In [ ]:
mode_indices = [7, 8]
true_coefs = [0.05, -0.2]
r = 1
nxy = 256

initial_guess = [0.1, -0.1]

mpsf = make_psf(r, true_coefs, mode_indices, nxy)
psf = make_psf(r, initial_guess, mode_indices, nxy)

# initialize
es = cma.CMAEvolutionStrategy(initial_guess, sigma0, opts)
# variable to store objective function values after each iteration
fit = np.empty(popsize)
while not es.stop():  # my_desired_target_f_value
    new_soln = es.ask()  
    for ct, soln in enumerate(new_soln):
        fit[ct] = objfunc(soln, mpsf, U, V)
    es.tell(new_soln, fit)
    es.disp(20)
    
xopt = es.result[0] 
objfuncopt = es.result[1]
print('Solution: ' + str(xopt))
print('Final objective function value: ' + str(objfuncopt))
print('Stopping criterion: ', es.stop())


<div class="alert alert-block alert-success">
<font color=black>

3. Show the plot of the simulated PSF from `make_psf` with the Zernike modes and coefficients you used. What is the coefficient you obtained from the optimization? What does the optimized or estimated PSF look like?

</font> 
</div>

<p style="text-align: left;" ><font size="+1"><b>System Identification in Real Data</b></p>

<div class="alert alert-block alert-warning">
<font color=black>
Real data are not as “clean” as simulated data. There are noise, detector non-idealities, and other aberrations we might not be modeling. Let us first get our model to be as close to the real system as possible.
</font> 
</div>

<div class="alert alert-block alert-success">
<font color=black>

1. For all data collection below, use the green fiter. 
    
    Calculate the image distance of your system. (Hint: you can used the magnification in lab 3 and the focal length of the lens in lab 1) <br/> Show your calculations below.
What is the image distance (`zi`)? Note that you have to solve for both object distance and image distance using magnification and focal length of the lens to get image distance (lecture notes from Lab 1).

Assign the `zi` variable in your `make_psf` function to the value calculated above. Also make sure the wavelength of light ($λ$) is correctly modeled.
</font> 
</div>

<div class="alert alert-block alert-warning">
<font color=black>

We don’t know the exact size of the exit pupil. Let’s estimate that first using **in-focus** data at wide open aperture.
</font> 
</div>

<div class="alert alert-block alert-success">
<font color=black>

2. Perform an estimation of the exit pupil size and the two Zernike tilt parameters (to ensure subpixel centering of your PSF). Paste below the measured PSF, estimated PSF, and the size of the aperture from your optimization. <br/><br/>
HINTS: This optimization is nonlinear and highly subject to local optima. 
- You will want to start with some good starting guesses in order to find the correct solution. 
- This includes a good guess of the exit pupil size and of the tilt parameters. 
- Recall that the tilts should be close to zero if your PSF is properly centered.
- The parameter `sigma0` in the `options` parameter passed to CMA-ES is a variance that controls the level of randomness in the search. If your search appears to get stuck far from an optima, you might want to increase this value.

</font> 
</div>

<div class="alert alert-block alert-success">
<font color=black>

3. Now, let’s consider a **defocused data set**. Select/collect a defocused PSF with a small to moderate level of defocus. Fix the aperture to the value you just calculated and estimate the level of defocus (and tilts, since they remain important). Show your measured PSF, estimated PSF, and state your estimated defocus.

</font> 
</div>

<div class="alert alert-block alert-success">
<font color=black>

4. Repeat the above estimation but include astigmatism, coma, and spherical aberration Zernikes. Show your measured PSF, estimated PSF, and state your estimated aberration parameters. Additionally, show the resulting pupil phase function (e.g. the weighted sum of these Zernikes that fits this data). Are these estimates different than the previous fit?
</font> 
</div>

<div class="alert alert-block alert-success">
<font color=black>

5. Repeat the above estimation once more but use all Zernike polynomials from 2-22. Show your measured PSF, estimated PSF, and state your estimated aberration parameters. Show the resulting pupil phase function. Are these estimates different than the previous ones? Lastly, show the pupil phase function for a perfectly focused system (i.e., zero out the defocus and x-,y-tilt terms and show the weighted sum of all other Zernike polynomials).
</font> 
</div>

<div class="alert alert-block alert-success">
<font color=black>

6. Collect an additional defocused data set (different than the one above). Try to reproduce the “1-wave” defocus case that you simulated previously where the center of the PSF is near-zero and the PSF has the form of an annulus (i.e., donut-shape). Conduct the estimation using Zernikes 2-22. Show the estimated pupil phase function. Also, show the pupil phase function for a perfectly focused system. How does this compare with the previous “perfect focus” estimate.
</font> 
</div>

<div class="alert alert-block alert-success">
<font color=black>

**BONUS #1** Repeat a defocus experiment using another wavelength filter. Show pupil phase functions and “perfect focus” estimates. How do these compare to previous estimates?
    
**BONUS #2** Estimating the Pupil Function in an Obstructed Imaging System

Many optical imaging systems do not have simple disc-shaped pupil functions. This is true in astronomical imaging devices including telescopes with central obstructions (typically to suspend a secondary mirror in the optical path, which allows a longer “b” distance in the lens equation (or “zi” more recently) while maintaining a compact optical instrument, effectively “folding” the optical path).
We will set up a microscope where a central obstruction has been placed in the pupil. Alter your make_psf code to allow for three addition parameters for the amplitude portion of your pupil function. Specifically, allow a disc-shaped central obstruction of unknown position (x,y) and unknown diameter. For the datasets that you acquire show measured and estimated PSFs, amplitude and phase of your pupil function, and estimated aberration parameters. (You may select Zernikes based on your previous characterizations of the microscope.)

</font> 
</div>